In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Question 1

In [108]:
df = pd.read_csv('w8q1.csv',index_col="Fruit_ID")

In [109]:
df

,weight,sweetness_level,Label
Fruit_ID,,,
1,180,7,Apple
2,200,6,Apple
3,150,4,Orange
4,170,5,Orange
5,160,6,Apple
6,140,3,Orange


In [110]:
w0 = 165
s0 = 5.5

In [111]:
euclidean_dist = np.array(np.sqrt((w0-df['weight'])**2 + (s0-df['sweetness_level'])**2))
euclidean_dist

array([15.07481343, 35.00357125, 15.07481343,  5.02493781,  5.02493781,
       25.12468905])

In [112]:
manhattan_dist = np.array(np.abs(w0-df['weight']) + np.abs(s0-df['sweetness_level']))
manhattan_dist

array([16.5, 35.5, 16.5,  5.5,  5.5, 27.5])

In [113]:
p = 4
minkowski_dist = np.array(((w0-df['weight'])**p + (s0-df['sweetness_level'])**p)**(1/p))
minkowski_dist

array([15.00037499, 35.00000036, 15.00037499,  5.000125  ,  5.000125  ,
       25.00062498])

In [118]:
# for euclidean distance
df["euclidean_dist"] = euclidean_dist
sorted_df = df.sort_values(by="euclidean_dist")
sorted_df

,weight,sweetness_level,Label,euclidean_dist
Fruit_ID,,,,
4,170,5,Orange,5.024938
5,160,6,Apple,5.024938
1,180,7,Apple,15.074813
3,150,4,Orange,15.074813
6,140,3,Orange,25.124689
2,200,6,Apple,35.003571


In [119]:
nearest_neighbors = sorted_df[:k]
predicted_value = nearest_neighbors['Label'].mode()[0]
print(f'Predicted value for 165 grams and 5.5 sweetness level : {predicted_value}')

Predicted value for 165 grams and 5.5 sweetness level : Orange


In [120]:
k = 1
nearest_neighbors = sorted_df[:k]
predicted_value = nearest_neighbors['Label'].mode()[0]
print(f'Predicted value for 165 grams and 5.5 sweetness level : {predicted_value}')

Predicted value for 165 grams and 5.5 sweetness level : Orange


In [121]:
k = 5
nearest_neighbors = sorted_df[:k]
predicted_value = nearest_neighbors['Label'].mode()[0]
print(f'Predicted value for 165 grams and 5.5 sweetness level : {predicted_value}')

Predicted value for 165 grams and 5.5 sweetness level : Orange


### Using sckit-learn

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
x = df[['weight', 'sweetness_level']]
y = df['Label']
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x, y)
y_pred = knn.predict(np.array([[165,5.5]]))[0]
print(f"Predicted value using scikit learn:{y_pred}")

Predicted value using scikit learn:Orange


/usr/lib/python3/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## Question 2

In [199]:
df = pd.read_csv("w8q2.csv",index_col = "Patient_ID")
df

,Age,Blood Pressure,Cholesterol,Diagnosis
Patient_ID,,,,
1,30,High,High,Sick
2,45,Low,Normal,Healthy
3,50,High,High,Sick
4,35,Low,Normal,Healthy
5,60,High,High,Sick
6,55,Low,Normal,Healthy
7,40,High,High,Sick
8,25,Low,Normal,Healthy
9,65,High,High,Sick


In [200]:
def entropy(column):
    n = len(column)
    p = column.value_counts(normalize=True)
    return -np.sum(p * np.log2(p))

In [201]:
target_entropy = entropy(df['Diagnosis'])
print(f'Entropy of Diagnosis: {target_entropy:.4f}')

Entropy of Diagnosis: 1.0000


In [202]:
def info_gain(df, feature, target):
    total_entropy = entropy(df[target])
    weighted_entropy = 0
    for value in df[feature].unique():
        subset = df[df[feature] == value]
        weighted_entropy += (len(subset) / len(df)) * entropy(subset[target])
    
    return total_entropy - weighted_entropy

In [203]:
features = ['Age','Blood Pressure','Cholesterol']
for feature in features:
    ig = info_gain(df, feature, 'Diagnosis')
    print(f'Information Gain for {feature}: {ig:.4f}')

Information Gain for Age: 1.0000
Information Gain for Blood Pressure: 1.0000
Information Gain for Cholesterol: 1.0000


In [204]:
info_gains = {feature: information_gain(df, feature, 'Diagnosis') for feature in features}
root_feature = max(info_gains, key=info_gains.get)
print(f"Root Node Feature: {root_feature} (Information Gain: {info_gains[root_feature]:.4f})")

Root Node Feature: Age (Information Gain: 1.0000)


In [205]:
decision_tree = {'feature': root_feature,'branches': {}}
for value in df[root_feature].unique():
    subset = df[df[root_feature] == value]
    diagnosis_counts = subset['Diagnosis'].value_counts()
    diagnosis = diagnosis_counts.idxmax()  # Take the most common diagnosis in this subset
    decision_tree['branches'][value] = {
        'label': diagnosis,
        'count': diagnosis_counts.to_dict(),
        'data': subset
    }

In [206]:
new_patient = {'Age': 50,'Blood Pressure': 'Low','Cholesterol': 'Normal'}

def predict(tree, patient):
    if 'branches' not in tree:
        return tree
    feature_value = patient[tree['feature']]
    if feature_value in tree['branches']:
        return predict(tree['branches'][feature_value], patient)
    else:
        return None
prediction = predict(decision_tree, new_patient)
print(f"\nPrediction for the new patient : {prediction}")


Prediction for the new patient : {'label': 'Sick', 'count': {'Sick': 1}, 'data':             Age Blood Pressure Cholesterol Diagnosis
Patient_ID                                          
3            50           High       High       Sick}


### Using Scikit-learn

In [224]:
from sklearn.tree import DecisionTreeClassifier, export_text
x = df[['Age', 'Blood Pressure', 'Cholesterol']]
y = df['Diagnosis']
x = pd.get_dummies(x, drop_first=True)

In [228]:
clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf.fit(x, y)
root_node_feature_index = clf.tree_.feature[0]
root_node_feature = x.columns[root_node_feature_index]
print(f"Root Node Feature: {root_node_feature}")
tree_rules = export_text(clf, feature_names=list(x.columns))
print("\nDecision Tree Rules:")
print(tree_rules)

Root Node Feature: Cholesterol_Normal

Decision Tree Rules:
|--- Cholesterol_Normal <= 0.50
|   |--- class: Sick
|--- Cholesterol_Normal >  0.50
|   |--- class: Healthy

